In [ ]:
epochs = 10
n_test_batches = 200

# Part 11 - 安全深度学习分类



## 您的数据很重要，您的模型也很重要

数据是机器学习的推动力 创建和收集数据的组织能够构建和训练自己的机器学习模型。这使他们能够向外部组织提供此类模型即服务（MLaaS）的使用。这对于某些组织很有用——他们无法自行创建这些模型，但仍希望使用此模型对自己的数据进行预测。

但是，托管在云中的模型仍然存在隐私/ IP问题。 为了让外部组织使用它——他们必须上传输入数据（例如要分类的图像）或下载模型。从隐私的角度来看，上传输入数据可能会出现问题，但是如果创建/拥有模型的组织不愿意，则下载模型可能不是一个选择。


## 计算加密数据

在这种情况下，一种潜在的解决方案是以一种方式对模型和数据进行加密，以允许一个组织使用另一组织拥有的模型，而无需将其IP彼此公开。存在几种允许对加密数据进行计算的加密方案，其中最为人熟知的类型是安全多方计算（SMPC），同态加密（FHE / SHE）和功能加密（FE）。我们将在这里集中讨论安全的多方计算（[在教程5的此处详细介绍](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%205%20-%20Intro% 20to％20Encrypted％20Programs.ipynb)），其中包含私有添加共享。它依赖于SecureNN和SPDZ等加密协议，[在此出色的博客文章中给出了详细信息](https://mortendahl.github.io/2017/09/19/private-image-analysis-with-mpc /)。

这些协议在加密数据上实现了卓越的性能，并且在过去的几个月中，我们一直在努力使这些协议易于使用。具体来说，我们正在构建工具，使您可以使用这些协议，而不必自己重新实现协议（甚至不必知道其工作原理背后的加密方法）。让我们进去看看。


## 设定

本教程中的确切设置如下：考虑您是服务器，并且有一些数据。首先，您使用此私人训练数据定义和训练模型。然后，您与拥有自己的一些数据的客户联系，该客户希望访问您的模型以做出一些预测。

您对模型（神经网络）进行加密。客户端加密其数据。然后，您都使用这两个加密资产来使用模型对数据进行分类。 最后，预测结果以加密方式发送回客户端，以便服务器（即您）对客户端数据一无所知（您既不了解输入也不了解预测）。

理想情况下，我们将在“服务器”与“客户端”之间共享输入，对模型亦然。为了简单起见，共享将由另外两个工作机“alice”和“ bob”持有。 如果您认为alice由客户端拥有，而bob由服务器拥有，也是一样的。

该计算在[许多MPC框架](https://arxiv.org/pdf/1801.03239.pdf)中是标准的半诚实（译者注：honest-but-curious，指的是遵循协议但会试图窃取隐私信息的敌手）敌手模型中是安全的。

**万事俱备, 我们开始吧！**


作者:
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel) · GitHub: [@LaRiffle](https://github.com/LaRiffle)

中文版译者：
- Hou Wei - github：[@dljgs1](https://github.com/dljgs1)

**Let's get started!**

### 导包以及模型规格

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

我们还需要执行特定于导入/启动PySyft的命令。我们创建了一些工作程序（分别命名为“client”，“bob”和“ alice”）。 最后，我们定义`crypto_provider`，它提供我们可能需要的所有加密原语（[请参阅我们在SMPC上的教程以了解更多详细信息](https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part ％2009％20-％20Intro％20to％20Encrypted％20Programs.ipynb)。

In [ ]:
import syft as sy
hook = sy.TorchHook(torch) 
client = sy.VirtualWorker(hook, id="client")
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider") 

我们定义学习任务的设置

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 50
        self.epochs = epochs
        self.lr = 0.001
        self.log_interval = 100

args = Arguments()

### 数据加载并发送给工作人员

在我们的设置中，我们假设服务器有权访问某些数据以首先训练其模型。这是MNIST训练集。

In [ ]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

其次，客户端具有一些数据，并且希望使用服务器的模型对其进行预测。该客户端通过在两个工作人员“alice”和“ bob”之间共享共享数据来加密其数据。
> SMPC使用要求在整数上工作的加密协议。我们在这里利用PySyft张量抽象来使用`.fix_precision()`将PyTorch浮点张量转换为固定精度张量。例如，精度为2的0.123在第二个十进制数字处进行舍入，因此存储的数字为整数12。


In [ ]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_precision().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_precision().share(alice, bob, crypto_provider=crypto_provider)
    ))

### 前馈神经网络规范
这是服务器使用的网络规范

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

### 启动训练
训练是在本地进行的，所以这是纯粹的本地PyTorch训练，这里没有什么特别的！

In [ ]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size,
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)


In [ ]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            output = F.log_softmax(output, dim=1)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
test(args, model, test_loader)

现在我们的模型已经过训练，可以随时提供服务！

## 安全评估

现在，作为服务器，我们将模型发送给持有数据的工作人员。由于模型是敏感信息（您已经花了很多时间优化它！），因此您不想透露其权重，因此像我们之前对数据集所做的那样，秘密共享模型。

In [ ]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

这个测试函数执行加密的评估。模型权重，数据输入，预测和用于评分的目标均已加密！

但是，语法与模型的纯PyTorch测试非常相似，不是很好吗？

我们从服务器端解密的唯一一件事是最后的最终分数，以验证预测的平均水平。

In [ ]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader[:n_test_batches]:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_total += args.test_batch_size
# 此“测试”功能执行加密的评估。 模型权重，数据输入，预测和评分目标均已加密！
# 但是，您可以看到，语法与普通的PyTorch测试非常相似！ 真好！

# 我们从服务器端解密的唯一一件事是在200批次数据的末尾最终得分，以验证预测的平均水平。      
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test set: Accuracy: {}/{} ({:.0f}%)'.format(
                n_correct, n_total,
                100. * n_correct / n_total))


In [ ]:
test(args, model, private_test_loader)

等等！ 在这里，您已经学会了如何进行端到端的安全预测：服务器模型的权重尚未泄漏到客户端，并且服务器没有有关数据输入或分类输出的信息！

关于性能，在我的笔记本电脑（2,7 GHz Intel Core i7，16GB RAM）上，对一张图像进行分类的时间**少于0.1秒，约33ms。**但是，这正在使用非常快速的通信（所有工作机都在我的本地计算机上）。 性能会因不同工作机之间的通信速度而异。

## 结论

您已经看到利用PyTorch和PySyft进行实用的安全机器学习并保护用户数据非常容易，而不必成为加密专家！

关于此主题的更多信息将很快推出，包括卷积层以相对于其他库正确地基准化PySyft性能，以及神经网络的私有加密训练，当组织依靠外部敏感数据来训练自己的模型时，这是必需的。 敬请关注！

如果您喜欢此方法，并希望加入保护隐私，去中心化AI和AI供应链（数据）所有权的运动，则可以通过以下方式做到这一点！

# 恭喜!!! 是时候加入社区了!

祝贺您完成本笔记本教程！ 如果您喜欢此方法，并希望加入保护隐私、去中心化AI和AI供应链（数据）所有权的运动，则可以通过以下方式做到这一点！

### 给 PySyft 加星

帮助我们的社区的最简单方法是仅通过给GitHub存储库加注星标！ 这有助于提高人们对我们正在构建的出色工具的认识。

- [Star PySyft](https://github.com/OpenMined/PySyft)

### 选择我们的教程

我们编写了非常不错的教程，以更好地了解联合学习和隐私保护学习的外观，以及我们如何为实现这一目标添砖加瓦。

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### 加入我们的 Slack!

保持最新进展的最佳方法是加入我们的社区！ 您可以通过填写以下表格来做到这一点[http://slack.openmined.org](http://slack.openmined.org)

### 加入代码项目!

对我们的社区做出贡献的最好方法是成为代码贡献者！ 您随时可以转到PySyft GitHub的Issue页面并过滤“projects”。这将向您显示所有概述，选择您可以加入的项目！如果您不想加入项目，但是想做一些编码，则还可以通过搜索标记为“good first issue”的GitHub问题来寻找更多的“一次性”微型项目。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 捐赠

如果您没有时间为我们的代码库做贡献，但仍想提供支持，那么您也可以成为Open Collective的支持者。所有捐款都将用于我们的网络托管和其他社区支出，例如黑客马拉松和聚会！

[OpenMined's Open Collective Page](https://opencollective.com/openmined)